In [58]:
import tensorflow as tf

import os
import pathlib
import time
import datetime

from keras import layers
from matplotlib import pyplot as plt
from IPython import display

### Path to Dataset

In [59]:
dataset_path = pathlib.Path(
    "./dataset"
)

for folder in dataset_path.iterdir():
    print(folder)

# Path to colored and grayscaled image folders
colored_path = dataset_path / "colored"
grayscaled_path = dataset_path / "grayscaled"

dataset\colored
dataset\grayscaled
dataset\test


### Fetch a sample image from the colored folder

In [60]:
sample_colored_image = list(colored_path.glob("*"))[0]
print(sample_colored_image)
image = tf.io.read_file(str(sample_colored_image))
image = tf.image.decode_image(image)
plt.figure()
# plt.imshow(image)
# plt.title("Sample Colored Image")
# plt.show()

dataset\colored\10.jpg


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

### Fetch a sample image from the grayscaled folder

In [77]:
sample_grayscaled_image = list(grayscaled_path.glob("*"))[0]  # Fetch the first image
print(sample_grayscaled_image)
image = tf.io.read_file(str(sample_grayscaled_image))
image = tf.image.decode_image(image)
# plt.figure()
# plt.imshow(image, cmap="gray")
# plt.title("Sample Grayscaled Image")
# plt.show()

dataset\grayscaled\10.jpg


### Loading the files and normalizing

In [89]:
def load(input_image_file, real_image_file):
    input_image = tf.io.read_file(input_image_file)
    input_image = tf.image.decode_jpeg(input_image)
    input_image = tf.cast(input_image, tf.float32)

    real_image = tf.io.read_file(real_image_file)
    real_image = tf.image.decode_jpeg(real_image)
    real_image = tf.cast(real_image, tf.float32)

    input_image = (input_image / 255.0)
    real_image = (real_image / 255.0)

    return input_image, real_image


# Tesing Load input and real images 
input_image, real_image = load("./dataset/grayscaled/10.jpg", "./dataset/colored/10.jpg")

### Cast images for visualization

In [91]:
# Plot the images
# plt.figure()
# plt.imshow(input_image, cmap="gray")
# plt.title("Input Image")
# plt.show()

# plt.figure()
# plt.imshow(real_image)
# plt.title("Real Image")
# plt.show()

### Skipping the "As described in the [pix2pix](https://arxiv.org/abs/1611.07004) paper, you need to apply random jittering and mirroring to preprocess the training set." part of preprocessing

### Fetch the file paths for grayscale and colored images


In [84]:
grayscale_images = tf.data.Dataset.list_files("./dataset/grayscaled/*.jpg")
colored_images = tf.data.Dataset.list_files("./dataset/colored/*.jpg")

paired_images = tf.data.Dataset.zip((grayscale_images, colored_images))
paired_images = paired_images.map(load)

BATCH_SIZE = 32
# Adjust buffer size as needed for shuffling (atleast > than your len(dataset)) in my case I have 5709 colored and grayscale images
BUFFER_SIZE = 6000  
paired_images = paired_images.shuffle(BUFFER_SIZE)
paired_images = paired_images.batch(BATCH_SIZE)